# Setup

### Libraries

In [45]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Janela

In [46]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

### Vertex e Fragment Shaders

In [47]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

In [48]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)




# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [49]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

# Matriz Model, View, Projection

In [50]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform) # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura, inc_fov, inc_near, inc_far
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

# Modelos e Texturas

In [51]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """

    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.strip().split() # quebra a linha por espaço
        if not values: continue

        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        elif values[0] == 'vn':
            pass

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []

            if len(values[1:]) == 3: # trigs
                for v in values[1:]:

                    w = v.split('/')
                    face.append(int(w[0]))
                    if len(w) >= 2 and len(w[1]) > 0:
                        face_texture.append(int(w[1]))
                    else:
                        face_texture.append(0)
                    
                faces.append((face, face_texture, material))
            elif len(values[1:]) == 4: # quads
                # print('reading quads')

                trig1 = [values[1], values[2], values[3]]
                trig2 = [values[3], values[4], values[1]]

                for v in trig1:
                    w = v.split('/')
                    face.append(int(w[0]))
                    if len(w) >= 2 and len(w[1]) > 0:
                        face_texture.append(int(w[1]))
                    else:
                        face_texture.append(0)

                for v in trig2:
                    w = v.split('/')
                    face.append(int(w[0]))
                    if len(w) >= 2 and len(w[1]) > 0:
                        face_texture.append(int(w[1]))
                    else:
                        face_texture.append(0)
               
                faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

In [52]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    print(img_textura,img.mode)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

In [53]:
vertices_list = []    
textures_coord_list = []

In [54]:
modelo = load_model_from_file('terreno/terreno.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo terreno.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo terreno.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(0,'terreno/terreno.png')

In [55]:
modelo = load_model_from_file('skybox/skybox.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo sky.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo sky.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(1,'skybox/skybox.png')

In [56]:
modelo = load_model_from_file('caminho/caminho.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo terreno.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo terreno.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(3,'caminho/caminho.jpg')

In [57]:
modelo = load_model_from_file('gato/gato.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cat.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(4,'gato/gato.jpg')

In [58]:
modelo = load_model_from_file('lobo/lobo.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cat.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(5,'lobo/lobo.jpg')

In [59]:
modelo = load_model_from_file('torre/torre.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cat.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(6,'torre/torre.jpg')

In [60]:
modelo = load_model_from_file('arvore/arvore.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo arvore.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo arvore.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(7,'arvore/casca.jpg')
load_texture_from_file(8,'arvore/folhas.png')

In [61]:
modelo = load_model_from_file('golem/golem.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cat.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(9,'golem/golem.tif')

In [62]:
modelo = load_model_from_file('gaveta/gaveta.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cat.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(10,'gaveta/gaveta.png')

In [63]:
modelo = load_model_from_file('caixa/caixa.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cube.obj. Vertice final:',len(vertices_list))

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(11,'caixa/caixa.jpg')

# Definição de Desenhos

In [64]:
def desenha_terreno():
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.01; t_z = 0.0;
    
    # escala
    s_x = 200.0; s_y = 200.0; s_z = 200.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 0)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 0, 6-0) ## renderizando

In [65]:
def desenha_sky(rotacao_inc):
    # rotacao
    angle = rotacao_inc;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 0.0; t_y = 0.0; t_z = 0.0;
    
    # escala
    s_x = 3; s_y = 3; s_z = 3;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 1)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 6, 5958-6) ## renderizando

In [66]:
def desenha_caminho():
    # rotacao
    angle = 0.0
    r_x = 0.0; r_y = 1.0; r_z = 0.0
    
    # translacao
    t_x = -54; t_y = -1.0; t_z = 0.0
    
    # escala
    s_x = 95; s_y = 1.0; s_z = 15
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 3)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 5958, 5964-5958) ## renderizando

In [67]:
def desenha_gato():
    # rotacao
    angle = 90.0;
    r_x = -1.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 30.0; t_y = 31.3; t_z = -3.7;
    
    # escala
    s_x = 0.05; s_y = 0.05; s_z = 0.05;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)

    loc_model = glGetUniformLocation(program, "model")

    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 4)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 5964, 217692-5964) ## renderizando

In [68]:
def desenha_lobo():
    # rotacao
    angle = -90.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0;
    
    # escala
    s_x = 12; s_y = 12; s_z = 12;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)

    loc_model = glGetUniformLocation(program, "model")

    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 5)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 217692, 225960-217692) ## renderizando

In [69]:
def desenha_torre():
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 30.0; t_y = -4.0; t_z = 0.0;
    
    # escala
    s_x = 5.0; s_y = 5.0; s_z = 5.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 6)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 225960, 232068-225960) ## renderizando

In [70]:
def desenha_arvore(x, z):
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    #t_x = -10.0; t_y = -1.0; t_z = 15.0;
    t_x = x; t_y = -1.0; t_z = z;
    
    # escala
    s_x = 7.0; s_y = 7.0; s_z = 7.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    ### desenho o tronco da arvore
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 7)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 232068, 252330-232068) ## renderizando
    
    ### desenho as folhas
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 8)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 252330, 273240-252330) ## renderizando

In [71]:
def desenha_golem():
    # rotacao
    angle = 90.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = -110; t_y = -1; t_z = 0.0;
    
    # escala
    s_x = 2.5; s_y = 2.5; s_z = 2.5;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)

    loc_model = glGetUniformLocation(program, "model")

    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 9)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 273240, 281637-273240) ## renderizando

In [72]:
def desenha_gaveta():
    # rotacao
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 33.5; t_y = 28.5; t_z = 2;
    
    # escala
    s_x = 1; s_y = 1; s_z = 1;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)

    loc_model = glGetUniformLocation(program, "model")

    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 10)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 281637, 313443-281637) ## renderizando

In [73]:
def desenha_caixa():
    # rotacao
    angle = 0.0;
    r_x = -1.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 30; t_y = 30; t_z = -4;
    
    # escala
    s_x = 1.2; s_y = 1.2; s_z = 1.2;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)

    loc_model = glGetUniformLocation(program, "model")

    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 11)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 313443, 313479-313443) ## renderizando

# Rodando

### Uso da GPU

In [74]:
buffer = glGenBuffers(2)

In [75]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

### Eventos

In [76]:
cameraPos   = glm.vec3(25.0,  33.0,  0.0)
cameraFront = glm.vec3(0.0,  0.0, -1.0)
cameraUp    = glm.vec3(0.0,  1.0,  0.0)

trap_mouse = True
polygonal_mode = False

inc_fov = 0
inc_near = 0
inc_far = 0
inc_view_up = 0

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, trap_mouse, inc_fov, inc_near, inc_far, cameraUp, inc_view_up
    
    cameraSpeed = 0.75
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1: polygonal_mode = not polygonal_mode # tecla P
            
    if key == 77 and action==1: trap_mouse = not trap_mouse # tecla M


firstMouse = True
yaw = 20.0 
pitch = 5.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;
    
    if pitch >= 89.9: pitch = 89.9
    if pitch <= -89.9: pitch = -89.9

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)
    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Execução Final

In [77]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

In [78]:
glEnable(GL_DEPTH_TEST)

rotacao_inc = 0
while not glfw.window_should_close(window):
    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode == True: glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
    else: glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)
        
    if trap_mouse == True: glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)
    else: glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_NORMAL)
    
    rotacao_inc += 0.1   
    desenha_sky(rotacao_inc)

    desenha_terreno()
    desenha_caminho()
    desenha_gato()
    desenha_lobo()
    desenha_torre()
    desenha_golem()
    desenha_gaveta()
    desenha_caixa()

    value = 5
    for i in range(7):
        desenha_arvore(value - 20*i, 20)
        desenha_arvore(value - 20*i, -20)
        desenha_arvore(value - 20*i, 40)
        desenha_arvore(value - 20*i, -40)
        desenha_arvore(value - 20*i, 60)
        desenha_arvore(value - 20*i, -60)
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    glfw.swap_buffers(window)

glfw.terminate()